In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
import holidays
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [112]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gdp = pd.read_csv('gdp.csv')

In [113]:
train_day = []
test_day = []
for i in range(len(train)):
    x = pd.to_datetime(train['date'][i])
    train_day.append(x.weekday())
    
train['weekday'] = train_day

for j in range(len(test)):
    y = pd.to_datetime(test['date'][j])
    test_day.append(y.weekday())
    
test['weekday'] = test_day

In [114]:
finland_holidays = holidays.FI()
norway_holidays = holidays.NO()
sweden_holidays = holidays.SE()
train_holiday = []
test_holiday = []
for i in range(len(train)):
    x = pd.to_datetime(train['date'][i])
    if (x in finland_holidays) or (x in norway_holidays) or (x in sweden_holidays):
        train_holiday.append(1)
    else:
        train_holiday.append(0)

train['holiday'] = train_holiday

for j in range(len(test)):
    y = pd.to_datetime(test['date'][j])
    if (y in finland_holidays) or (y in norway_holidays) or (y in sweden_holidays):
        test_holiday.append(1)
    else:
        test_holiday.append(0)

test['holiday'] = test_holiday

In [115]:
train = train.set_index('row_id')
test = test.set_index('row_id')

In [116]:
y_train = train['num_sold']
train = train.drop('num_sold',axis=1)

In [117]:
train.head()

,date,country,store,product,weekday,holiday
row_id,,,,,,
0,2015-01-01,Finland,KaggleMart,Kaggle Mug,3,1
1,2015-01-01,Finland,KaggleMart,Kaggle Hat,3,1
2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,3,1
3,2015-01-01,Finland,KaggleRama,Kaggle Mug,3,1
4,2015-01-01,Finland,KaggleRama,Kaggle Hat,3,1


In [118]:
gdp

,year,GDP_Finland,GDP_Norway,GDP_Sweden
0,2015,234.440,385.802,505.104
1,2016,240.608,368.827,515.655
2,2017,255.017,398.394,541.019
3,2018,275.580,437.000,555.455
4,2019,268.782,405.510,533.880


In [119]:
train['country'].value_counts()

Norway     8766
Sweden     8766
Finland    8766
Name: country, dtype: int64

In [120]:
train['store'].value_counts()

KaggleMart    13149
KaggleRama    13149
Name: store, dtype: int64

In [121]:
train['product'].value_counts()

Kaggle Hat        8766
Kaggle Sticker    8766
Kaggle Mug        8766
Name: product, dtype: int64

In [122]:
date = np.array(train['date'])
test_date = np.array(test['date'])

year_test,month_test,day_test = [],[],[]
year,month,day = [],[],[]

for i in range(len(date)):
    lst = date[i].split('-')
    year.append(int(lst[0]))
    month.append(int(lst[1]))
    day.append(int(lst[2]))  
    
for i in range(len(test_date)):
    lst = test_date[i].split('-')
    year_test.append(int(lst[0]))
    month_test.append(int(lst[1]))
    day_test.append(int(lst[2]))

In [123]:
train = train.drop('date',axis=1)
test = test.drop('date',axis=1)

train['year'] = year
train['month'] = month
train['day'] = day

test['year'] = year_test
test['month'] = month_test
test['day'] = day_test

In [124]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [125]:
gdp_train = []
for i in range(len(train)):
    if train['year'][i] == 2015:
        if train['country_Finland'][i] == 1:
            gdp_train.append(gdp['GDP_Finland'].iloc[0])
        if train['country_Norway'][i] == 1:
            gdp_train.append(gdp['GDP_Norway'].iloc[0])
        if train['country_Sweden'][i] == 1:
            gdp_train.append(gdp['GDP_Sweden'].iloc[0])
            
    if train['year'][i] == 2016:
        if train['country_Finland'][i] == 1:
            gdp_train.append(gdp['GDP_Finland'].iloc[1])
        if train['country_Norway'][i] == 1:
            gdp_train.append(gdp['GDP_Norway'].iloc[1])
        if train['country_Sweden'][i] == 1:
            gdp_train.append(gdp['GDP_Sweden'].iloc[1])
            
    if train['year'][i] == 2017:
        if train['country_Finland'][i] == 1:
            gdp_train.append(gdp['GDP_Finland'].iloc[2])
        if train['country_Norway'][i] == 1:
            gdp_train.append(gdp['GDP_Norway'].iloc[2])
        if train['country_Sweden'][i] == 1:
            gdp_train.append(gdp['GDP_Sweden'].iloc[2])
            
    if train['year'][i] == 2018:
        if train['country_Finland'][i] == 1:
            gdp_train.append(gdp['GDP_Finland'].iloc[3])
        if train['country_Norway'][i] == 1:
            gdp_train.append(gdp['GDP_Norway'].iloc[3])
        if train['country_Sweden'][i] == 1:
            gdp_train.append(gdp['GDP_Sweden'].iloc[3])
            
    if train['year'][i] == 2019:
        if train['country_Finland'][i] == 1:
            gdp_train.append(gdp['GDP_Finland'].iloc[4])
        if train['country_Norway'][i] == 1:
            gdp_train.append(gdp['GDP_Norway'].iloc[4])
        if train['country_Sweden'][i] == 1:
            gdp_train.append(gdp['GDP_Sweden'].iloc[4])
            
train['GDP'] = gdp_train                        

In [126]:
train.head()

,weekday,holiday,year,month,day,country_Finland,country_Norway,country_Sweden,store_KaggleMart,store_KaggleRama,product_Kaggle Hat,product_Kaggle Mug,product_Kaggle Sticker,GDP
row_id,,,,,,,,,,,,,,
0,3,1,2015,1,1,1,0,0,1,0,0,1,0,234.44
1,3,1,2015,1,1,1,0,0,1,0,1,0,0,234.44
2,3,1,2015,1,1,1,0,0,1,0,0,0,1,234.44
3,3,1,2015,1,1,1,0,0,0,1,0,1,0,234.44
4,3,1,2015,1,1,1,0,0,0,1,1,0,0,234.44


In [127]:
test = test.reset_index()
gdp_test = []
for i in range(len(test)):
    if train['country_Finland'][i] == 1:
        gdp_test.append(gdp['GDP_Finland'].iloc[4])
    if train['country_Norway'][i] == 1:
        gdp_test.append(gdp['GDP_Norway'].iloc[4])
    if train['country_Sweden'][i] == 1:
        gdp_test.append(gdp['GDP_Sweden'].iloc[4])
        
test['GDP'] = gdp_test
test = test.set_index('row_id')

In [128]:
test.head()

,weekday,holiday,year,month,day,country_Finland,country_Norway,country_Sweden,store_KaggleMart,store_KaggleRama,product_Kaggle Hat,product_Kaggle Mug,product_Kaggle Sticker,GDP
row_id,,,,,,,,,,,,,,
26298,1,1,2019,1,1,1,0,0,1,0,0,1,0,268.782
26299,1,1,2019,1,1,1,0,0,1,0,1,0,0,268.782
26300,1,1,2019,1,1,1,0,0,1,0,0,0,1,268.782
26301,1,1,2019,1,1,1,0,0,0,1,0,1,0,268.782
26302,1,1,2019,1,1,1,0,0,0,1,1,0,0,268.782


In [129]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [130]:
from keras import backend as K
def tf_smape(y_true, y_pred):
    yt = tf.cast(y_true,tf.float64)
    yp = tf.cast(y_pred,tf.float64)
    return tf.abs(yt - yp) / (yt + tf.abs(yp)) * 200
    
    

In [131]:
def get_params(df):
    mini = min(df['mean_test_score'])
    for row in range(len(df)):
        if df.iloc[row]['mean_test_score'] == mini:
            return df.iloc[row]['params']

**Random Forest Model**

In [68]:
from sklearn.ensemble import RandomForestRegressor

In [70]:
scorer = make_scorer(smape, greater_is_better=True)

clf = RandomizedSearchCV(RandomForestRegressor(),{
    'n_estimators' : [100,1000,10000],
    'min_samples_split' : [7,8],
    'min_samples_leaf' : [3,4], 
    'max_depth' : [None,3,4,5]
}, scoring = scorer,return_train_score=False)

clf.fit(train,y_train)

new_df = pd.DataFrame(clf.cv_results_)

In [71]:
min(new_df['mean_test_score'])

6.981408789098713

In [72]:
get_params(new_df)

{'n_estimators': 10000,
 'min_samples_split': 7,
 'min_samples_leaf': 3,
 'max_depth': None}

In [107]:
rf = RandomForestRegressor(n_estimators=10000,
                          min_samples_split=7,
                          min_samples_leaf=3)

rf.fit(train,y_train)
preds = rf.predict(test)

In [74]:
preds 

array([ 394.72695078,  546.68543234,  161.64736331, ...,  967.00402004,
       1443.73043725,  386.69919149])

In [75]:
test = test.reset_index()
submission = pd.DataFrame(columns=['row_id','num_sold'])
submission['row_id'] = test['row_id']
submission['num_sold'] = preds

In [76]:
submission.to_csv('submission4.csv',index=False)

**Regression Model**

In [76]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

In [73]:
scorer = make_scorer(smape, greater_is_better=True)

clf_reg = GridSearchCV(LinearRegression(),{
    'fit_intercept' : [True,False]   
}, scoring = scorer,return_train_score=False)

clf_reg.fit(train,y_train)

new_df_reg = pd.DataFrame(clf_reg.cv_results_)

In [74]:
min(new_df_reg['mean_test_score'])

32.58612197888383

In [75]:
get_params(new_df_reg)

{'fit_intercept': True}

In [82]:
scorer = make_scorer(smape, greater_is_better=True)

clf_ridge = GridSearchCV(Ridge(),{
    'alpha' : [1,1.5,2,2.5,3,3.5,4],
    'fit_intercept' : [True,False],
    'solver' : ['auto','svd','cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}, scoring = scorer,return_train_score=False)

clf_ridge.fit(train,y_train)

new_df_ridge = pd.DataFrame(clf_ridge.cv_results_)

In [83]:
min(new_df_ridge['mean_test_score'])

26.142898378202016

In [84]:
get_params(new_df_ridge)

{'alpha': 2.5, 'fit_intercept': False, 'solver': 'sag'}

**Boosting Models**

In [64]:
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor
from lightgbm import LGBMRegressor

In [87]:
scorer = make_scorer(smape, greater_is_better=True)

clf_ada = RandomizedSearchCV(AdaBoostRegressor(n_estimators=100),{
    'learning_rate' : [1,2,3,4],
    'loss' : ['linear', 'square', 'exponential']
}, scoring = scorer,return_train_score=False)

clf_ada.fit(train,y_train)

new_df_ada = pd.DataFrame(clf_ada.cv_results_)

In [88]:
min(new_df_ada['mean_test_score'])

61.57088584557596

In [89]:
get_params(new_df_ada)

{'loss': 'linear', 'learning_rate': 1}

In [97]:
scorer = make_scorer(smape, greater_is_better=True)

clf_cat = RandomizedSearchCV(CatBoostRegressor(iterations=1000,verbose=0),{
    'depth' : [2,3,4,5],
    'learning_rate' : [0.01,0.1,0.2,0.5],
    'random_strength' : [1.0,2.0,3.0,4.0],
    'min_data_in_leaf' : [2,3,4,5]
    
}, scoring = scorer,return_train_score=False)

clf_cat.fit(train,y_train,verbose=None)

new_df_cat = pd.DataFrame(clf_cat.cv_results_)

In [98]:
min(new_df_cat['mean_test_score'])

7.266216869431174

In [99]:
get_params(new_df_cat)

{'random_strength': 4.0,
 'min_data_in_leaf': 2,
 'learning_rate': 0.1,
 'depth': 5}

In [158]:
preds = (np.sum(predictions, axis=0)) / len(predictions)

In [159]:
preds

array([ 283.18718958,  454.87681835,  137.05661281, ...,  745.31918084,
       1040.75736045,  384.0562636 ])

In [160]:
test = test.reset_index()
submission = pd.DataFrame(columns=['row_id','num_sold'])
submission['row_id'] = test['row_id']
submission['num_sold'] = preds

In [161]:
submission.to_csv('submission4.csv',index=False)

In [100]:
scorer = make_scorer(smape, greater_is_better=True)

clf_xgb = RandomizedSearchCV(xgb.XGBRegressor(n_estimators=1000,verbosity=0),{
    'learning_rate' : [1,1.5,2],
    'gamma' : [0.5,1,1.5],
    'reg_lambda' : [0.5,1,1.5]
    
}, scoring = scorer,return_train_score=False)

clf_xgb.fit(train,y_train,verbose=None)

new_df_xgb = pd.DataFrame(clf_xgb.cv_results_)

In [101]:
min(new_df_xgb['mean_test_score'])

12.54735549158194

In [102]:
get_params(new_df_xgb)

{'reg_lambda': 0.5, 'learning_rate': 1, 'gamma': 1}

In [54]:
preds = (np.sum(predictions, axis=0)) / len(predictions)

In [56]:
test = test.reset_index()
submission = pd.DataFrame(columns=['row_id','num_sold'])
submission['row_id'] = test['row_id']
submission['num_sold'] = preds

In [57]:
submission.to_csv('submission4.csv',index=False)

In [77]:
scorer = make_scorer(smape, greater_is_better=True)

clf_lgbm = RandomizedSearchCV(LGBMRegressor(random_state=0),{
    
    'n_estimators': [175,250,500,1000,2000],
    'max_depth': [10,11,12],
    'learning_rate': [0.05, 0.075, 0.1, 0.125, 0.15]
}, scoring = scorer,return_train_score=False)

clf_lgbm.fit(train,y_train,verbose=None)

new_df_lgbm = pd.DataFrame(clf_lgbm.cv_results_)

In [78]:
min(new_df_lgbm['mean_test_score'])

6.596692447945225

In [79]:
get_params(new_df_lgbm)

{'n_estimators': 250, 'max_depth': 11, 'learning_rate': 0.125}

In [ ]:
params0 = {'objective': 'regression', # Manual optimization
           'force_row_wise': True,
           'max_bin': 400, # need more bins than days in a year
           'verbosity': -1,
           'seed': 1,
           'bagging_seed': 3,
           'feature_fraction_seed': 2,
           'learning_rate': 0.018,
           'lambda_l1': 0,
           'lambda_l2': 1e-2,
           'num_leaves': 18,
           'feature_fraction': 0.710344827586207,
           'bagging_fraction': 0.47931034482758617,
           'bagging_freq': 3,
           'min_child_samples': 20}

In [132]:
lgbm_mod = LGBMRegressor(
    objective = 'regression', # Manual optimization
    force_row_wise = True,
    max_bin = 400, # need more bins than days in a year
    verbosity = -1,
    seed = 1,
    bagging_seed = 3,
    feature_fraction_seed = 2,
    learning_rate = 0.018,
    lambda_l1 = 0,
    lambda_l2 = 1e-2,
    num_leaves = 18,
    feature_fraction = 0.710344827586207,
    bagging_fraction = 0.47931034482758617,
    bagging_freq = 3,
    min_child_samples = 20)

lgbm_mod.fit(train,y_train)
preds = lgbm_mod.predict(test)

In [133]:
preds

array([255.96822032, 419.08231906, 170.8934609 , ..., 606.03723794,
       979.88960798, 245.27842942])

In [134]:
test = test.reset_index()
submission = pd.DataFrame(columns=['row_id','num_sold'])
submission['row_id'] = test['row_id']
submission['num_sold'] = preds

In [135]:
submission.to_csv('submission4.csv',index=False)

**Keras Model**

In [42]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [43]:
train.shape

(26298, 14)

In [44]:
label_normalizer = tf.keras.layers.Normalization(axis=-1)
feature_normalizer = layers.Normalization(input_shape=[14,], axis=None)

In [45]:
feature_normalizer.adapt(train)
label_normalizer.adapt(y_train)

In [80]:
tf_model = tf.keras.Sequential([
    feature_normalizer,
    layers.Dense(14,activation='exponential'),
    layers.Dense(5,activation='exponential'),
    layers.Dense(1,activation='exponential')
])

tf_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_3 (Normalizat  (None, 14)               3         
 ion)                                                            
                                                                 
 dense_19 (Dense)            (None, 14)                210       
                                                                 
 dense_20 (Dense)            (None, 5)                 75        
                                                                 
 dense_21 (Dense)            (None, 1)                 6         
                                                                 
Total params: 294
Trainable params: 291
Non-trainable params: 3
_________________________________________________________________


In [81]:
tf_model.compile(
    optimizer=tf.optimizers.Adamax(learning_rate=0.01),
    loss=tf_smape)

In [83]:
history = tf_model.fit(
    train,
    y_train,
    epochs=5000,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [84]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,val_loss,epoch
4995,NaN,NaN,4995
4996,NaN,NaN,4996
4997,NaN,NaN,4997
4998,NaN,NaN,4998
4999,NaN,NaN,4999


In [40]:
preds = tf_model.predict(test)

In [41]:
test = test.reset_index()
submission = pd.DataFrame(columns=['row_id','num_sold'])
submission['row_id'] = test['row_id']
submission['num_sold'] = preds

In [42]:
submission.to_csv('submission4.csv',index=False)